In [1]:
from maverick import Game, Player, ActionType, GameState, GameEventType, PlayerState, GameStateType
import pprint

In [2]:
class CallBot(Player):
    """A passive bot that always calls or checks."""

    def decide_action(
        self,
        game_state: GameState,
        valid_actions: list[ActionType],
        min_raise: int
    ) -> tuple[ActionType, int]:
        """Always call or check if possible, otherwise fold."""
        if ActionType.CHECK in valid_actions:
            return (ActionType.CHECK, 0)
        elif ActionType.CALL in valid_actions:
            call_amount = game_state.current_bet - self.current_bet
            if call_amount <= self.stack:
                return (ActionType.CALL, call_amount)
        return (ActionType.FOLD, 0)

In [3]:
class AggressiveBot(Player):
    """An aggressive bot that frequently bets and raises."""

    def decide_action(
        self,
        game_state: GameState,
        valid_actions: list[ActionType],
        min_raise: int
    ) -> tuple[ActionType, int]:
        """Bet or raise aggressively."""
        # Try to raise if possible
        if ActionType.RAISE in valid_actions:
            raise_amount = min_raise
            if raise_amount <= self.stack:
                return (ActionType.RAISE, raise_amount)

        # Otherwise bet if possible
        if ActionType.BET in valid_actions:
            bet_amount = game_state.big_blind * 2
            if bet_amount <= self.stack:
                return (ActionType.BET, bet_amount)

        # Call if we can't bet/raise
        if ActionType.CALL in valid_actions:
            call_amount = game_state.current_bet - self.current_bet
            if call_amount <= self.stack:
                return (ActionType.CALL, call_amount)

        # Check if possible
        if ActionType.CHECK in valid_actions:
            return (ActionType.CHECK, 0)

        # Otherwise fold
        return (ActionType.FOLD, 0)

In [4]:
class SmartBot(Player):
    """A strategic bot that considers hand strength."""

    def decide_action(
        self,
        game_state: GameState,
        valid_actions: list[ActionType],
        min_raise: int
    ) -> tuple[ActionType, int]:
        """Make strategic decisions based on game state."""
        # Simple strategy: aggressive early, cautious later
        pot_size = game_state.pot

        # On pre-flop, be selective
        if len(game_state.community_cards) == 0:
            if ActionType.CHECK in valid_actions:
                return (ActionType.CHECK, 0)
            elif ActionType.CALL in valid_actions:
                call_amount = game_state.current_bet - self.current_bet
                # Only call small bets pre-flop
                if call_amount <= game_state.big_blind * 3:
                    if call_amount <= self.stack:
                        return (ActionType.CALL, call_amount)
            return (ActionType.FOLD, 0)

        # Post-flop, consider pot size
        if ActionType.BET in valid_actions and pot_size < game_state.big_blind * 10:
            bet_amount = pot_size // 2
            if bet_amount <= self.stack and bet_amount >= game_state.min_bet:
                return (ActionType.BET, bet_amount)

        if ActionType.CHECK in valid_actions:
            return (ActionType.CHECK, 0)

        if ActionType.CALL in valid_actions:
            call_amount = game_state.current_bet - self.current_bet
            # Use pot odds to decide
            if call_amount <= pot_size // 3 and call_amount <= self.stack:
                return (ActionType.CALL, call_amount)

        return (ActionType.FOLD, 0)

In [5]:
# Create game with blinds
game = Game(small_blind=10, big_blind=20, max_hands=40)

# Create and add players with different strategies
players = [
    CallBot(id="p1", name="CallBot", stack=1000, seat=0),
    AggressiveBot(id="p2", name="AggroBot", stack=1000, seat=1),
    SmartBot(id="p3", name="SmartBot", stack=1000, seat=2),
]

for player in players:
    game.add_player(player)
    
game.start_game()

In [6]:
game.get_game_info()

{'state': 'PRE_FLOP',
 'street': 'PRE_FLOP',
 'hand_number': 41,
 'pot': 30,
 'current_bet': 20,
 'community_cards': [],
 'dealer_index': 0,
 'button_position': 0,
 'players': [{'id': 'p1',
   'name': 'CallBot',
   'stack': 1620,
   'state': 'ACTIVE',
   'position': 'BUTTON',
   'current_bet': 20},
  {'id': 'p3',
   'name': 'SmartBot',
   'stack': 1350,
   'state': 'ACTIVE',
   'position': None,
   'current_bet': 10}]}